Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression with regularization

In [5]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularizer = tf.scalar_mul(beta, tf.nn.l2_loss(weights))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.663763
Minibatch accuracy: 4.7%
Validation accuracy: 10.4%
Minibatch loss at step 500: 3.197729
Minibatch accuracy: 74.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.761219
Minibatch accuracy: 77.3%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 1.175112
Minibatch accuracy: 79.7%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 1.103854
Minibatch accuracy: 78.9%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.749824
Minibatch accuracy: 83.6%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.667683
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Test accuracy: 88.4%


### DNN with regularization

In [7]:
batch_size = 128
nnodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nnodes]))
  b1 = tf.Variable(tf.zeros([nnodes]))
  
  w2 = tf.Variable(
    tf.truncated_normal([nnodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf.nn.relu_layer(tf_train_dataset, w1, b1), w2) + b2
  regularizer = tf.scalar_mul(beta, tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regularizer)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu_layer(tf_valid_dataset, w1, b1), w2) + b2
  )
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu_layer(tf_test_dataset, w1, b1), w2) + b2
  )

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 782.030151
Minibatch accuracy: 10.2%
Validation accuracy: 39.6%
Minibatch loss at step 500: 194.509155
Minibatch accuracy: 75.8%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 116.925583
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 68.870583
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 41.529884
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.266727
Minibatch accuracy: 85.9%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.379475
Minibatch accuracy: 89.8%
Validation accuracy: 86.4%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

restricted_train_dataset = train_dataset[:500, :]
restricted_train_labels = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 717.588501
Minibatch accuracy: 9.4%
Validation accuracy: 39.9%
Minibatch loss at step 500: 190.127319
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 1000: 115.303650
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 1500: 69.926529
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2000: 42.407364
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2500: 25.718681
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 3000: 15.598721
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Test accuracy: 82.0%


The model learns the training dataset perfectly neglecting its capacity to generalize.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128
nnodes = 1024
beta = 0.001
dropout_keep_prob = 0.5  # Drop half of the weights

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nnodes]))
  b1 = tf.Variable(tf.zeros([nnodes]))
  
  w2 = tf.Variable(
    tf.truncated_normal([nnodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  layer1 = tf.matmul(tf_train_dataset, w1) + b1
  relu_layer1 = tf.nn.relu(layer1)
  dropout_layer1 = tf.nn.dropout(relu_layer1, keep_prob=dropout_keep_prob)
    
  logits = tf.matmul(dropout_layer1, w2) + b2
  regularizer = tf.scalar_mul(beta, tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regularizer)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu_layer(tf_valid_dataset, w1, b1), w2) + b2
  )
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu_layer(tf_test_dataset, w1, b1), w2) + b2
  )

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 811.270142
Minibatch accuracy: 10.2%
Validation accuracy: 38.6%
Minibatch loss at step 500: 201.541977
Minibatch accuracy: 68.0%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 119.516174
Minibatch accuracy: 74.2%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 70.290253
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 42.311829
Minibatch accuracy: 75.8%
Validation accuracy: 82.9%
Minibatch loss at step 2500: 25.378033
Minibatch accuracy: 85.2%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 15.509835
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Test accuracy: 91.4%


### ... overfiting case

In [12]:
num_steps = 3001

restricted_train_dataset = train_dataset[:500, :]
restricted_train_labels = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 745.081421
Minibatch accuracy: 11.7%
Validation accuracy: 32.6%
Minibatch loss at step 500: 194.032898
Minibatch accuracy: 99.2%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 116.283020
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 70.542755
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 42.786137
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 25.950516
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 3000: 15.743503
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 84.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


**Architecture**
* 6 layers with decreasing number of hidden units (power of 2)
* RELu activation functions
* Dropout of 50% in all layers

**Regularization:**
* L2 Regularization. Alpha = 0.001

**Optimization:**
* SGD with Adaptive learning rate with exponential decay. Starts at 0.5
* 10000 steps


In [5]:
import numpy as np
import math 

n_hidden_layers = 6
batch_size = 128
beta = 0.001
dropout_keep_prob = 0.5  # Drop half of the weights
start_learning_rate = 0.5

nnodes = np.cumprod([1024] + [0.5 for i in range(n_hidden_layers -1)]).astype(int)
print(nnodes)

[1024  512  256  128   64   32]


In [6]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    w0 = tf.Variable(
        tf.truncated_normal([image_size * image_size, nnodes[0]],  
                            stddev=math.sqrt(2.0/(image_size*image_size)))
    )
    b0 = tf.Variable(tf.zeros([nnodes[0]]))

    w1 = tf.Variable(
        tf.truncated_normal([nnodes[0], nnodes[1]], 
                            stddev=math.sqrt(2.0/(nnodes[0])))
    )
    b1 = tf.Variable(tf.zeros([nnodes[1]]))

    w2 = tf.Variable(
        tf.truncated_normal([nnodes[1], nnodes[2]], 
                            stddev=math.sqrt(2.0/(nnodes[1])))
    )
    b2 = tf.Variable(tf.zeros([nnodes[2]]))

    w3 = tf.Variable(
        tf.truncated_normal([nnodes[2], nnodes[3]], 
                            stddev=math.sqrt(2.0/(nnodes[2])))
    )
    b3 = tf.Variable(tf.zeros([nnodes[3]]))

    w4 = tf.Variable(
        tf.truncated_normal([nnodes[3], nnodes[4]], 
                            stddev=math.sqrt(2.0/(nnodes[3])))
    )
    b4 = tf.Variable(tf.zeros([nnodes[4]]))
    
    w5 = tf.Variable(
        tf.truncated_normal([nnodes[4], nnodes[5]], 
                            stddev=math.sqrt(2.0/(nnodes[4])))
    )
    b5 = tf.Variable(tf.zeros([nnodes[5]]))

    w6 = tf.Variable(
        tf.truncated_normal([nnodes[5], num_labels], 
                            stddev=math.sqrt(2.0/(nnodes[5])))
    )
    b6 = tf.Variable(tf.zeros([num_labels]))
    
    # Connect layers.
    # We multiply the inputs with the weight matrix, and add biases. 
    layer0_logits = tf.matmul(tf_train_dataset, w0) + b0
    layer0_hidden = tf.nn.relu(layer0_logits)
    layer0_dropout = tf.nn.dropout(layer0_hidden, keep_prob=dropout_keep_prob)

    layer1_logits = tf.matmul(layer0_dropout, w1) + b1
    layer1_hidden = tf.nn.relu(layer1_logits)
    layer1_dropout = tf.nn.dropout(layer1_hidden, keep_prob=dropout_keep_prob)

    layer2_logits = tf.matmul(layer1_dropout, w2) + b2
    layer2_hidden = tf.nn.relu(layer2_logits)
    layer2_dropout = tf.nn.dropout(layer2_hidden, keep_prob=dropout_keep_prob)

    layer3_logits = tf.matmul(layer2_dropout, w3) + b3
    layer3_hidden = tf.nn.relu(layer3_logits)
    layer3_dropout = tf.nn.dropout(layer3_hidden, keep_prob=dropout_keep_prob)

    layer4_logits = tf.matmul(layer3_dropout, w4) + b4
    layer4_hidden = tf.nn.relu(layer4_logits)
    layer4_dropout = tf.nn.dropout(layer4_hidden, keep_prob=dropout_keep_prob)
    
    layer5_logits = tf.matmul(layer4_dropout, w5) + b5
    layer5_hidden = tf.nn.relu(layer5_logits)
    layer5_dropout = tf.nn.dropout(layer5_hidden, keep_prob=dropout_keep_prob)
    
    logits = tf.matmul(layer5_dropout, w6) + b6

    # L2 Regularization.
    regularizers = ( 
        tf.nn.l2_loss(w0) +
        tf.nn.l2_loss(w1) + 
        tf.nn.l2_loss(w2) + 
        tf.nn.l2_loss(w3) + 
        tf.nn.l2_loss(w4) +
        tf.nn.l2_loss(w5) +
        tf.nn.l2_loss(w6)
    )

    # Training computation.
    # We compute the cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Addaptive learning rate. Source [https://www.tensorflow.org/api_docs/python/tf/train/exponential_decay]
    # Initialize counter variable for the number of steps taken
    global_step = tf.Variable(0, trainable=False)  
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    # Passing global_step to minimize() will increment it at each step.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training data.
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for the validation data.
    valid_prediction = tf.nn.softmax(
    tf.nn.relu_layer(
        tf.nn.relu_layer(
            tf.nn.relu_layer(
                tf.nn.relu_layer(
                    tf.nn.relu_layer(
                        tf.nn.relu_layer(
                            tf.nn.relu_layer(
                                tf_valid_dataset,
                                w0, b0), 
                            w1, b1), 
                        w2, b2), 
                    w3, b3), 
                w4, b4),
            w5, b5), 
        w6, b6)
    )
    
    # Predictions for the test data.
    test_prediction = tf.nn.softmax(
    tf.nn.relu_layer(
        tf.nn.relu_layer(
            tf.nn.relu_layer(
                tf.nn.relu_layer(
                    tf.nn.relu_layer(
                        tf.nn.relu_layer(
                            tf.nn.relu_layer(
                                tf_test_dataset,
                                w0, b0), 
                            w1, b1), 
                        w2, b2), 
                    w3, b3), 
                w4, b4),
            w5, b5), 
        w6, b6)
    )


In [7]:
num_steps = 10000


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 2000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.235620
Minibatch accuracy: 14.1%
Validation accuracy: 11.2%
Minibatch loss at step 5000: 1.127113
Minibatch accuracy: 77.3%
Validation accuracy: 83.1%
Test accuracy: 91.0%
